# Custom Hitster
With this notebook you can generate your own hitster version based on a Spotify playlist. Follow these steps to create a double sided printable PDF with custom Hitster cards:

- Step 1: Create a playlist on Spotify.
- Step 2: Export information about the playlist as a CSV file using [Exportify](https://exportify.net/).
- Step 3: Run the cell below by clicking on the triangle button on the left. You will be asked to upload the CSV file as downloaded from Exportify.
- Step 4: Download the resulting _custom_hitster.pdf_ file from the folder icon on the left.


Tips:
- Try to make a playlist with variation in release years to make the game more interesting.
- The resulting PDF is in A4 format.


In [ ]:
#@title Click the triangle on the left to start


################################################################
# Upload CSV file for playlist
################################################################
print('----> Upload your CSV file as downloaded from Exportify...')

from google.colab import files
import shutil

# Upload the file
uploaded = files.upload()

# Fix file name
old_file_name = list(uploaded.keys())[0]
new_file_name = 'playlist_info.csv'
shutil.copyfile(old_file_name, new_file_name)

print('----> Upload your CSV file as downloaded from Exporitify: DONE')

################################################################
# Install packages
################################################################
print('\n\n----> Installing Libraries...')
!pip install segno
!pip install reportlab
print('----> Installing Libraries: DONE')


################################################################
# Read and preprocess the data
################################################################
print('\n\n----> Preprocessing data set')
import pandas as pd
from datetime import date

df = pd.read_csv('playlist_info.csv')

# Extract release year from dates
def extract_year(full_date):
  return full_date[:4]

df['year'] = df['Release Date'].apply(extract_year).astype(int)

# Rename the dataframe columns
df['artist'] = df['Artist Name(s)']
df['track_id'] = df['Track ID']
df['track_name'] = df['Track Name']

print('----> Preprocessing data set: DONE')

################################################################
# Create the cards using reportlab library
################################################################

# First define some helper functions

################################################################
# Function for drawing song information on canvas
################################################################
def draw_text(c, text, the_text_size, x, y, font, sub_page_width):
    """
    Adds multi-line text to the canvas, wrapping the text if it exceeds the sub_page_width.

    Parameters:
    - c: Canvas object
    - text: Text to be added
    - the_text_size: Font size of the text
    - x, y: Center coordinates of the text
    - font: Font to be used
    - sub_page_width: Maximum width allowed for the text
    """
    c.setFont(font, the_text_size)

    # Split text into words
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        # Measure width of current line with the new word added
        potential_line = f"{current_line} {word}".strip()
        potential_width = c.stringWidth(potential_line, font, the_text_size)

        if potential_width <= sub_page_width:
            # Add word to the current line
            current_line = potential_line
        else:
            # Start a new line
            lines.append(current_line)
            current_line = word

    # Append the last line
    if current_line:
        lines.append(current_line)

    # Calculate total height of the text block
    line_height = the_text_size * 1.2
    total_text_height = len(lines) * line_height

    # Adjust the starting y-coordinate to vertically center the text block
    start_y = y + (total_text_height / 2) - (line_height * 0.5)

    # Draw each line
    for i, line in enumerate(lines):
        text_width = c.stringWidth(line, font, the_text_size)
        text_x = x - (0.5 * text_width)
        text_y = start_y - (i * line_height)
        c.drawString(text_x, text_y, line)


################################################################
# Function to generate the fronts of the cards
################################################################
import random
import numpy as np
from reportlab.lib.colors import black

def generate_front(c, rows, page_info):
  width, height, width_step, height_step, bg_margin = page_info

  # Loop over each card on page
  track_ids = []
  i = 0
  for h in np.arange(0, height, height_step):
    for w in np.arange(0, width, width_step):

      # Stop if all track were handled already
      if i >= len(rows):
        return track_ids

      # Get values
      year = rows['year'].values[i]
      artist_name = rows['artist'].values[i]
      song = rows['track_name'].values[i]
      id = rows['track_id'].values[i]
      track_ids.append(id)

      # Draw outlines
      c.setLineWidth(bg_margin * 2)
      c.setStrokeColor(black)
      c.rect(w, h, width_step, height_step, stroke=1, fill=0)

      # Draw background
      sub_page_width = width_step - 2*bg_margin
      bg = random.choice(backgrounds)
      c.drawImage(
          bg,
          w + bg_margin,
          h + bg_margin,
          width=sub_page_width,
          height=height_step - 2*bg_margin
      )

      # Draw texts
      #    Year
      text_x = w + 0.5*width_step
      text_y = h + 0.4*height_step

      draw_text(
          c, str(year), 45,
          text_x, text_y,
          'Helvetica-Bold', sub_page_width
      )

      #    Artist
      text_x = w + 0.5*width_step
      text_y = h + 0.8*height_step

      draw_text(
          c, artist_name, 14,
          text_x, text_y,
          'Helvetica-Bold', sub_page_width
      )

      #    Song
      text_x = w + 0.5*width_step
      text_y = h + 0.17*height_step

      draw_text(
          c, song, 12,
          text_x, text_y,
          'Helvetica-Oblique', sub_page_width
      )

      i += 1

  return track_ids


################################################################
# Function to mirror horizontally to align QR with front side for double-sided-printing
################################################################
def horizontally_mirror(the_list):
  """
    The PDF should be double-sided printable.
    To make sure each QR code aligns correctly with the song info on the front,
    the list should be horizontally mirrrored.
  """
  chunks = [list(reversed(the_list[x:x+4])) for x in range(0, len(the_list), 4)]

  flipped = []
  for chunk in chunks:
    for el in chunk:
      flipped.append(el)

  return flipped

################################################################
# Function to generate backs of cards
################################################################
from reportlab.lib.colors import Color


def generate_back(c, track_ids, page_info, qr_bg):
  width, height, width_step, height_step, bg_margin = page_info
  sub_page_width = width_step - 2*bg_margin

  # Mirror horizontally to align QR with front side for double-sided-printing
  track_ids = horizontally_mirror(track_ids)

  i = 0
  for h in np.arange(0, height, height_step):
    for w in np.arange(0, width, width_step):

      # Stop if all tracks have been handled already
      if i >= len(track_ids):
        return

      # Get values
      track_id = track_ids[i]

      # Draw background image
      c.drawImage(qr_bg,
            w + bg_margin,
            h + bg_margin,
            width=sub_page_width,
            height=height_step - 2*bg_margin
          )

      # Load QR Code
      qr_image = ImageReader(f'./qr/{track_id}.png')

      # Draw QR Code
      c.drawImage(qr_image,
                  w + (0.25 * width_step) + bg_margin,
                  h + (0.25 * height_step),
                  width=sub_page_width/2,
                  height=height_step/2
                )

      # Draw outlines
      c.setLineWidth(bg_margin*2)  # Thickness of 2 units
      border_color = Color(0 / 255.0, 255 / 255.0, 255 / 255.0)
      c.setStrokeColor(border_color)
      c.rect(w, h, width_step, height_step, stroke=1, fill=0)

      # Increment index
      i += 1


################################################################
# Generate all QR Codes
################################################################
print('\n\n----> Generating QR codes...')

import segno
from tqdm import tqdm
import os

def make_url(track_id):
  return f'https://open.spotify.com/track/{track_id}'

def make_qr(id, url):
  qr = segno.make_qr(url)
  qr.save(f'./qr/{id}.png', scale=5, border=1)

if not os.path.exists('qr'):
  os.mkdir('qr')

for id in tqdm(df['track_id'].values):
  url = make_url(id)
  make_qr(id, url)

print('----> Generating QR codes: DONE')

################################################################
# Load background images
################################################################
print('\n\n----> Loading background images...')

from reportlab.lib.utils import ImageReader
import urllib.request

# Download QR background
if not os.path.isdir('qr'):
  os.mkdir('qr')

urllib.request.urlretrieve("https://github.com/felix-olivier/robster/blob/master/bg/qr_bg.jpg?raw=true", "qr_bg.jpg")

# Download front backgrounds
if not os.path.isdir('bg'):
  os.mkdir('bg')

for file_name in ['bg', 'bg2', 'bg3', 'bg4', 'bg5', 'bg6']:
  file_name += '.png'

  url = f'https://github.com/felix-olivier/robster/blob/master/bg/front/{file_name}?raw=true'
  urllib.request.urlretrieve(url, f'./bg/{file_name}')

# Load files for reportlab
backgrounds = []
for file in os.listdir('bg'):
  if file.endswith('.png'):
    img = ImageReader(f'./bg/{file}')
    backgrounds.append(img)

print('----> Loading background images: DONE')

################################################################
# Loop over all songs and generate cards
################################################################
print('\n\n----> Creating document...')

import math
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4


# Set up Canvas
c = canvas.Canvas('custom_hister.pdf', pagesize=A4)
width, height = A4
width_step = width / 4
height_step = height / 5
bg_margin = 5
page_info = (width, height, width_step, height_step, bg_margin)

# Calculate number of pages
cards_per_page = int((width / width_step) * (height / height_step))
nr_pages = int(math.ceil(len(df) / 20))

# Load bg image qr
qr_bg = ImageReader(f'./qr_bg.jpg')


# Construct the document
for i in tqdm(range(0, nr_pages)):

  # Extract relevant rows from dataframe
  start = i * cards_per_page
  end = min(start + cards_per_page, len(df))
  rows = df[start:end]

  # Add fronts of cards
  track_ids = generate_front(c, rows, page_info)

  # Add page
  c.showPage()

  # Add Backs of cards
  generate_back(c, track_ids, page_info, qr_bg)

  # Add page
  c.showPage()


# save pdf
c.save()

print('----> Creating document: DONE')
print('-----------------> Download the file custom_hitster.pdf from the menu on the left')